In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
from shapely.geometry import Point, Polygon, MultiPolygon
from pyproj import Proj, transform
ox.config(use_cache=True, log_console=True)
ox.__version__

### let's get polygons of some certain areas and add them to a dataframe

In [ ]:
place_names = ['Danubyu, Ayeyarwady Region, Myanmar', 
               'Gyobingauk, Bago Region, Myanmar',
               'Katha, Shan Region, Myanmar']
# https://www.citypopulation.de/Myanmar-Cities.html

In [ ]:
small_cities = ox.gdf_from_places(place_names, gdf_name='small_cities')
small_cities

### Evaluating if a certain point is inside a polygon

In [ ]:
# Shwe Yaung Pya Sedi
small_cities["geometry"][0].contains(Point(95.585605, 17.271707))
# anecdote time :-)

In [ ]:
# project the geometry to the appropriate UTM zone then plot it
small_cities = ox.project_gdf(small_cities)
fig, ax = ox.plot_shape(small_cities)

In [ ]:
# danubyu
small_cities["geometry"][0]

In [ ]:
print(small_cities["geometry"][0])